<a href="https://colab.research.google.com/github/sarbajeetroy/your_AI_assistant/blob/main/Your_AI_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U crawl4ai
!playwright install
!pip install -r requirements.txt

  Using cached crawl4ai-0.6.3-py3-none-any.whl.metadata (36 kB)
Using cached crawl4ai-0.6.3-py3-none-any.whl (292 kB)
  Attempting uninstall: crawl4ai
    Found existing installation: Crawl4AI 0.4.247
    Uninstalling Crawl4AI-0.4.247:
      Successfully uninstalled Crawl4AI-0.4.247
Removing unused browser at /root/.cache/ms-playwright/chromium-1169
Removing unused browser at /root/.cache/ms-playwright/chromium_headless_shell-1169
Removing unused browser at /root/.cache/ms-playwright/ffmpeg-1011
Removing unused browser at /root/.cache/ms-playwright/firefox-1482
Removing unused browser at /root/.cache/ms-playwright/webkit-2158
161.3 MiB [] 0% 0.0s161.3 MiB [] 0% 5.7s161.3 MiB [] 0% 3.5s161.3 MiB [] 1% 2.9s161.3 MiB [] 2% 2.5s161.3 MiB [] 3% 2.2s161.3 MiB [] 4% 2.0s161.3 MiB [] 4% 8.4s161.3 MiB [] 5% 8.2s161.3 MiB [] 5% 8.1s161.3 MiB [] 5% 7.7s161.3 MiB [] 6% 7.2s161.3 MiB [] 7% 6.6s161.3 MiB [] 8% 6.1s161.3 MiB [] 9% 5.4s161.3 MiB [] 10% 5.1s161.3 MiB [] 10% 5.0s161.3 MiB [] 11% 4.9s161

In [ ]:
!pip install langchain-core
!pip install langchain-google-genai
!pip install langchain-chroma

from datetime import datetime, timedelta
from typing import List, Dict, Any, Optional
import os
import logging
logging.basicConfig(level=logging.ERROR)

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
from termcolor import colored

In [ ]:
import os
import sys
import json
import asyncio
import requests
from xml.etree import ElementTree
from typing import List, Dict, Any
from dataclasses import dataclass
from datetime import datetime, timezone
from urllib.parse import urlparse
from dotenv import load_dotenv

from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig, CacheMode
from openai import AsyncOpenAI

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings


load_dotenv()

# Set Google API Key
google_api_key = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = google_api_key  # Add your API key here

# Initialize Gemini LLM and Embeddings
LLM = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.7, max_output_tokens=1500)
embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


# Initialize ChromaDB
# You can specify a directory for persistence
chroma_client = Chroma(collection_name="crawled_data", embedding_function=embeddings_model)


@dataclass
class ProcessedChunk:
    url: str
    chunk_number: int
    title: str
    summary: str
    content: str
    metadata: Dict[str, Any]
    embedding: List[float]

def chunk_text(text: str, chunk_size: int = 5000) -> List[str]:
    """Split text into chunks, respecting code blocks and paragraphs."""
    chunks = []
    start = 0
    text_length = len(text)

    while start < text_length:
        # Calculate end position
        end = start + chunk_size

        # If we're at the end of the text, just take what's left
        if end >= text_length:
            chunks.append(text[start:].strip())
            break

        # Try to find a code block boundary first (```)
        chunk = text[start:end]
        code_block = chunk.rfind('```')
        if code_block != -1 and code_block > chunk_size * 0.3:
            end = start + code_block

        # If no code block, try to break at a paragraph
        elif '\n\n' in chunk:
            # Find the last paragraph break
            last_break = chunk.rfind('\n\n')
            if last_break > chunk_size * 0.3:  # Only break if we're past 30% of chunk_size
                end = start + last_break

        # If no paragraph break, try to break at a sentence
        elif '. ' in chunk:
            # Find the last sentence break
            last_period = chunk.rfind('. ')
            if last_period > chunk_size * 0.3:  # Only break if we're past 30% of chunk_size
                end = start + last_period + 1

        # Extract chunk and clean it up
        chunk = text[start:end].strip()
        if chunk:
            chunks.append(chunk)

        # Move start position for next chunk
        start = max(start + 1, end)

    return chunks

async def get_title_and_summary(chunk: str, url: str) -> Dict[str, str]:
    """Extract title and summary using LLM (Gemini)."""
    system_prompt = """You are an expert AI assistant specialized in analyzing and summarizing technical documentation. Your primary task is to process a given text chunk from documentation and extract a concise title and a brief, informative summary."""

    try:
        response = await LLM.ainvoke(
            f"URL: {url}\n\nContent:\n{chunk[:2000]}...\n\n{system_prompt}"  # Increased chunk size for LLM context
        )
        # Assuming the LLM response is a string containing the JSON object
        # Need to parse the JSON string from the response text
        response_text = response.content.strip()
        # Clean up the response text to ensure it's valid JSON
        if response_text.startswith("```json"):
            response_text = response_text[7:]
        if response_text.endswith("```"):
            response_text = response_text[:-3]
        return json.loads(response_text)

    except Exception as e:
        print(f"Error getting title and summary: {e}")
        return {"title": "Error processing title", "summary": "Error processing summary"}


async def get_embedding(text: str) -> List[float]:
    """Get embedding vector from Gemini embeddings model."""
    try:
        # The GoogleGenerativeAIEmbeddings.aembed_query returns a list containing a single embedding
        embedding_response = await embeddings_model.aembed_query(text)
        return embedding_response
    except Exception as e:
        print(f"Error getting embedding: {e}")
        # Return a list of zeros with the expected embedding dimension for the model
        # The 'models/embedding-001' model has an embedding dimension of 768
        return [0] * 768


async def process_and_store_document_chroma(url: str, markdown: str, chroma_collection):
    """Process a document and store its chunks in ChromaDB."""
    # Split into chunks
    chunks = chunk_text(markdown)

    # Process chunks and create LangChain Documents
    documents = []
    for i, chunk in enumerate(chunks):
        # Get title and summary
        extracted = await get_title_and_summary(chunk, url)

        # Create metadata
        metadata = {
            "source": url,
            "chunk_number": i,
            "title": extracted.get('title', 'No Title'),
            "summary": extracted.get('summary', 'No Summary'),
            "url_path": urlparse(url).path
        }

        # Create a LangChain Document
        document = Document(
            page_content=chunk,
            metadata=metadata
        )
        documents.append(document)

    # Add documents to ChromaDB collection
    if documents:
        try:
            chroma_collection.add_documents(documents)
            print(f"Added {len(documents)} documents to ChromaDB for {url}")
        except Exception as e:
            print(f"Error adding documents to ChromaDB: {e}")

async def crawl_parallel_chroma(urls: List[str], chroma_collection, max_concurrent: int = 5):
    """Crawl multiple URLs in parallel with a concurrency limit and store in ChromaDB."""
    browser_config = BrowserConfig(
        headless=True,
        verbose=False,
        extra_args=["--disable-gpu", "--disable-dev-shm-usage", "--no-sandbox"],
    )
    crawl_config = CrawlerRunConfig(cache_mode=CacheMode.BYPASS)

    # Create the crawler instance
    crawler = AsyncWebCrawler(config=browser_config)
    await crawler.start()

    try:
        # Create a semaphore to limit concurrency
        semaphore = asyncio.Semaphore(max_concurrent)

        async def process_url(url: str):
            async with semaphore:
                result = await crawler.arun(
                    url=url,
                    config=crawl_config,
                    session_id="session1" # Use a consistent session ID if needed for caching
                )
                if result.success:
                    print(f"Successfully crawled: {url}")
                    await process_and_store_document_chroma(url, result.markdown_v2.raw_markdown, chroma_collection)
                else:
                    print(f"Failed: {url} - Error: {result.error_message}")

        # Process all URLs in parallel with limited concurrency
        await asyncio.gather(*[process_url(url) for url in urls])
    finally:
        await crawler.close()

def get_site_ai_docs_urls() -> List[str]:
    """Get URLs from sitemap."""
    sitemap_url = "https://www.your_site/sitemap.xml"
    try:
        response = requests.get(sitemap_url)
        response.raise_for_status()

        # Parse the XML
        root = ElementTree.fromstring(response.content)

        # Extract all URLs from the sitemap
        namespace = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}
        urls = [loc.text for loc in root.findall('.//ns:loc', namespace)]

        return urls
    except Exception as e:
        print(f"Error fetching sitemap: {e}")
        return []

async def main():
    # Get URLs from Sitemap for AI docs
    urls = get_site_ai_docs_urls()
    if not urls:
        print("No URLs found to crawl")
        return

    print(f"Found {len(urls)} URLs to crawl")
    await crawl_parallel_chroma(urls, chroma_client)

if __name__ == "__main__":
    # asyncio.run(main()) # Remove this line
    await main() # Await the main function directly

Found 4 URLs to crawl
Successfully crawled: https://www.thomsonreuters.in/en.html
Successfully crawled: https://www.thomsonreuters.in/en/about-us.html
Successfully crawled: https://www.thomsonreuters.in/en/about-us/leadership-team.html
Successfully crawled: https://www.thomsonreuters.in/en/about-us/locations.html
Added 3 documents to ChromaDB for https://www.thomsonreuters.in/en/about-us.html
Added 3 documents to ChromaDB for https://www.thomsonreuters.in/en/about-us/locations.html
Added 3 documents to ChromaDB for https://www.thomsonreuters.in/en/about-us/leadership-team.html
Added 4 documents to ChromaDB for https://www.thomsonreuters.in/en.html


In [ ]:
from langchain_chroma import Chroma
import os

# Define a directory to save the database
persist_directory = "./chroma_db"

# Ensure the directory exists
if not os.path.exists(persist_directory):
    os.makedirs(persist_directory)

# Initialize ChromaDB with persistence
chroma_client = Chroma(
    collection_name="crawled_data",
    embedding_function=embeddings_model, # Assuming embeddings_model is already defined
    persist_directory=persist_directory
)

print(f"ChromaDB initialized and will persist to {persist_directory}")

ChromaDB initialized and will persist to ./chroma_db


In [ ]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings # Import if needed for the embedding function

# Define the directory where the database is saved
persist_directory = "./chroma_db"

# Re-initialize the embeddings model if it's needed for querying
# embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001") # Uncomment if embeddings_model is not in your current session

# Load the ChromaDB from the persistent directory
chroma_client = Chroma(
    collection_name="crawled_data",
    embedding_function=embeddings_model, # Use the same embedding function
    persist_directory=persist_directory
)

print(f"ChromaDB loaded from {persist_directory}")

# You can now interact with the loaded data, e.g., perform a similarity search
# results = chroma_client.similarity_search("your query here")
# print(results)

ChromaDB loaded from ./chroma_db


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Define the prompt template for the RAG model
template = """You are an expert AI assistant for Thomson Reuters India, with comprehensive knowledge about the company's services, products, and information available on the Thomson Reuters India website (https://www.thomsonreuters.in/).

Your primary role is to provide accurate, helpful information about Thomson Reuters India to website visitors. You have access to all the documentation, content, and resources from the Thomson Reuters India website to help you answer user queries effectively.

Don't ask the user before taking an action, just respond with the most relevant information. Always make sure you look at the available content before answering the user's question unless you're certain of the answer.

When searching for information, always start with the "About Us" section to understand the context of the query. Then check other relevant sections of the website to provide comprehensive answers.

Always provide factual information directly from Thomson Reuters India's content. If a user asks about topics unrelated to Thomson Reuters India, politely redirect them to ask questions about the company, its services, products, or related information.

If you don't know the answer or can't find relevant information in the available content, be honest about it. You can suggest that the user contact Thomson Reuters India directly through the contact information provided on the website.

Your responses should reflect Thomson Reuters' professional tone while being helpful, clear, and concise.

Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Create a retriever from the ChromaDB collection
retriever = chroma_client.as_retriever()

# Create the RAG chain
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | LLM
    | StrOutputParser()
)

# Example usage:
question = "What is Thomson Reuters India about?"
response = rag_chain.invoke(question)
print(response)

Thomson Reuters India plays a crucial role in the global operations of Thomson Reuters. It serves as a dynamic hub, contributing significantly to product development, content management, and customer support. The India team collaborates with colleagues worldwide to deliver innovative solutions and services to professionals in the legal, tax, accounting, compliance, government, and media sectors.
